# Creación del dataset NG-IIoTset

In [11]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import ipaddress
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from collections import Counter

## Definición de funciones

### create_ng_iiotset()
Funcion que crea NG-IIoTset: Una Nueva Generación de conjunto de datos de seguridad para IIoT
basado en Edge-IIoTset pero con una distribución personalizada

In [20]:
def create_ng_iiotset():

    # 1. Definición de la distribución personalizada (manteniendo ±5% de variación respecto al original)

    target_distribution = {
        'normal': 10662000,               # Ligeramente reducido respecto al original
        'backdoor': 30000,                # Aumentado para tener más representación
        'ddos_http': 240000,              # Ligeramente aumentado
        'ddos_icmp': 2850000,             # Ligeramente reducido
        'ddos_tcp_syn': 2100000,          # Ligeramente aumentado
        'ddos_udp': 3100000,              # Ligeramente reducido
        'os_fingerprint': 5000,           # Aumentado para mejor representación
        'mitm_arp_dns': 6000,             # Aumentado para mejor representación
        'password': 1000000,              # Prácticamente igual
        'port_scan': 45000,               # Duplicado para dar énfasis
        'ransomware': 25000,              # Aumentado por relevancia actual
        'sql_injection': 60000,           # Ligeramente aumentado
        'upload': 50000,                  # Aumentado
        'vuln_scan': 150000,              # Prácticamente igual
        'xss': 25000                      # Aumentado por relevancia
    }
    
    # 2. Procesar todos los archivos Zeek en una única pasada
    all_data = {}
    
    # Definir las columnas a extraer para cada tipo de log
    log_columns = {
        'conn': ['uid', 'ts', 'id.orig_h', 'id.resp_h', 'id.resp_p', 'proto', 'service',
                 'conn_state', 'duration', 'orig_bytes', 'resp_bytes', 'orig_pkts',
                 'resp_pkts', 'ip_proto'],
        'dns': ['uid', 'query', 'answers', 'qtype_name', 'rcode', 'rcode_name'],
        'mqtt_connect': ['uid', 'connect_status', 'client_id'],
        'mqtt_publish': ['uid', 'topic', 'payload'],
        'modbus': ['uid', 'func', 'pdu_type', 'exception'],
        'http': ['uid', 'method', 'uri', 'user_agent', 'host',
                 'request_body_len', 'response_body_len', 'status_code'],
        'files': ['uid', 'fuid', 'source', 'mime_type', 'filename',
                 'seen_bytes', 'total_bytes', 'md5', 'sha1', 'sha256'],
        'weird': ['uid', 'name'],
        
    }
    
    # Procesar primero los archivos conn.log ya que contienen la mayor parte de información
    print("Procesando archivos conn.log...")
    base_df = process_conn_logs(normal_logs_dir, attack_logs_dir, log_columns['conn'])
    
    if base_df.empty:
        print("Error: No se pudieron procesar los archivos conn.log")
        return None
    
    print(f"Base DataFrame creada con {len(base_df)} registros")
    
    # Procesar los demás logs y unirlos al DataFrame base
    for log_type in ['dns', 'mqtt_connect', 'mqtt_publish', 'modbus', 'http', 'files', 'weird']:
        print(f"Procesando archivos {log_type}.log...")
        log_df = process_log_type(log_type, normal_logs_dir, attack_logs_dir, log_columns[log_type])
        
        if not log_df.empty:
            print(f"  - Encontrados {len(log_df)} registros, fusionando...")
            # Fusionar por uid
            base_df = pd.merge(base_df, log_df, on='uid', how='left')
    
    # 3. Aplicar muestreo estratificado personalizado
    print("Aplicando muestreo estratificado personalizado...")
    final_df = pd.DataFrame()
    
    for attack_type, target_count in target_distribution.items():
        subset = base_df[base_df['typeAttack'] == attack_type]
        print(f"  - {attack_type}: {len(subset)} registros originales, objetivo {target_count}")
        
        if len(subset) == 0:
            print(f"    ¡Advertencia! No hay registros del tipo {attack_type}")
            continue
        
        # Estrategia de muestreo
        if len(subset) > target_count:
            # Si hay más registros de los necesarios, realizar muestreo
            sampled = subset.sample(n=target_count, random_state=42)
            print(f"    Reduciendo mediante muestreo a {len(sampled)} registros")
        else:
            # Si hay menos, duplicar registros hasta alcanzar el objetivo
            factor = target_count // len(subset)
            remainder = target_count % len(subset)
            
            if factor > 1:
                # Duplicar registros
                duplicated = pd.concat([subset] * factor)
                # Añadir los registros restantes
                remaining = subset.sample(n=remainder, random_state=42)
                sampled = pd.concat([duplicated, remaining])
                print(f"    Aumentando mediante duplicación a {len(sampled)} registros")
            else:
                # Suficientes para el objetivo o cercano
                sampled = subset
                print(f"    Manteniendo los {len(sampled)} registros originales")
        
        final_df = pd.concat([final_df, sampled])
    
    # 4. Limpieza final
    print("Realizando limpieza final...")
    # Eliminar columnas duplicadas que puedan haberse generado en las uniones
    
    # Rellenar valores nulos
    for col in final_df.columns:
        if final_df[col].dtype == 'object':
            final_df[col] = final_df[col].fillna('unknown')
        else:
            final_df[col] = final_df[col].fillna(0)
    
    # Resetear índices
    final_df = final_df.reset_index(drop=True)
    
    print(f"Dataset personalizado creado con {len(final_df)} registros")
    return final_df

#### process_conn_logs
Función que procesa todos los archivos conn.log para crear el DataFrame base

In [21]:
def process_conn_logs(normal_dir, attack_dir, columns_to_keep):
    
    normal_files = glob.glob(f"{normal_dir}/**/conn.log", recursive=True)
    attack_files = glob.glob(f"{attack_dir}/**/conn.log", recursive=True)
    
    dfs = []
    
    # Procesar archivos de tráfico normal
    for log_file in tqdm(normal_files, desc="Archivos normales"):
        df = read_zeek_log(log_file, columns_to_keep)  # Pasa las columnas a mantener
        if df is not None and not df.empty:
            df['isAttack'] = 0
            df['typeAttack'] = 'normal'
            dfs.append(df)
    
    # Procesar archivos de ataques
    for log_file in tqdm(attack_files, desc="Archivos de ataque"):
        attack_type = determine_attack_type(log_file)
        df = read_zeek_log(log_file, columns_to_keep)  # Pasa las columnas a mantener
        
        if df is not None and not df.empty:
            df['isAttack'] = 1
            df['typeAttack'] = attack_type
            dfs.append(df)
    
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    return pd.DataFrame()

#### process_log_type
Función que procesa el resto de archivos.log

In [22]:
def process_log_type(log_type, normal_dir, attack_dir, columns):

    normal_files = glob.glob(f"{normal_dir}/**/{log_type}.log", recursive=True)
    attack_files = glob.glob(f"{attack_dir}/**/{log_type}.log", recursive=True)
    
    dfs = []
    
    # Procesar archivos
    for log_file in normal_files + attack_files:
        df = read_zeek_log(log_file, columns)
        if df is not None and not df.empty:
            dfs.append(df)
    
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    return pd.DataFrame()

#### read_zeek_log
Funcion que lee los archivos de LOG de Zeek y extrae las columnas especificadas

In [23]:
def read_zeek_log(log_file, columns_to_keep=None):

    try:
        # Primero leemos los encabezados para obtener los nombres de las columnas
        headers = None
        separator = None
        
        with open(log_file, 'r', encoding='utf-8', errors='replace') as f:
            for line in f:
                if line.startswith('#fields'):
                    headers = line.strip().split('\t')[1:]
                    break
                if line.startswith('#separator'):
                    separator_value = line.strip().split(' ')[1]
                    if separator_value.startswith('\\x'):
                        separator = bytes.fromhex(separator_value[2:]).decode('utf-8')
                    else:
                        separator = separator_value.encode().decode('unicode_escape')
        
        if headers is None or separator is None:
            return None
        
        # Leer el archivo con pandas
        df = pd.read_csv(log_file, 
                         comment='#', 
                         sep=separator, 
                         names=headers, 
                         na_values=['-'],
                         quoting=3,
                         error_bad_lines=False,
                         warn_bad_lines=False,
                         low_memory=False,
                         encoding='utf-8')
        
        # Filtrar columnas si es necesario
        if columns_to_keep is not None:
            # Mantener solo columnas que existen en el DataFrame
            cols_to_use = [col for col in columns_to_keep if col in df.columns]
            df = df[cols_to_use]
        
        return df
    
    except Exception as e:
        print(f"Error al leer {log_file}: {str(e)}")
        return None

#### determine_attack_type
Funcion que determina el tipo de ataque a partir de la ruta del archivo

In [24]:
def determine_attack_type(file_path):

    attack_mapping = {
        "Backdoor_attack": "backdoor",
        "DDoS HTTP Flood": "ddos_http",
        "DDoS ICMP Flood": "ddos_icmp",
        "DDoS TCP SYN Flood": "ddos_tcp_syn",
        "DDoS UDP Flood": "ddos_udp",
        "MITM": "mitm_arp_dns",
        "OS Fingerprinting": "os_fingerprint",
        "Password": "password",
        "Port Scanning": "port_scan",
        "Ransomware": "ransomware",
        "SQL injection": "sql_injection",
        "Uploading": "upload",
        "Vulnerability scanner": "vuln_scan",
        "XSS": "xss"
    }
    
    for key, value in attack_mapping.items():
        if key in file_path:
            return value
    
    # Si no se encuentra correspondencia, extraer el nombre de la carpeta
    parts = file_path.split(os.sep)
    for part in parts:
        for key, value in attack_mapping.items():
            if key.lower() in part.lower():
                return value
    
    return "unknown_attack"

## Ejecución

**Rutas de los directorios**

In [25]:
normal_logs_dir = "../Zeek-Pipeline/zeek_logs/normal"
attack_logs_dir = "../Zeek-Pipeline/zeek_logs/ataques"

**Crear el dataset personalizado y guardarlo como "NG-IIoTset.csv"**

In [26]:
ng_iiotset_df = create_ng_iiotset()
ng_iiotset_df.to_csv("NG-IIoTset.csv", index=False)


Procesando archivos conn.log...


Archivos de ataque: 100%|██████████| 14/14 [00:12<00:00,  1.13it/s]


Base DataFrame creada con 7649572 registros
Procesando archivos dns.log...
  - Encontrados 24790 registros, fusionando...
Procesando archivos mqtt_connect.log...
  - Encontrados 666556 registros, fusionando...
Procesando archivos mqtt_publish.log...
  - Encontrados 666555 registros, fusionando...
Procesando archivos modbus.log...
  - Encontrados 98087 registros, fusionando...
Procesando archivos http.log...
  - Encontrados 196008 registros, fusionando...
Procesando archivos files.log...
  - Encontrados 282551 registros, fusionando...
Procesando archivos weird.log...
  - Encontrados 1176213 registros, fusionando...
Aplicando muestreo estratificado personalizado...
  - normal: 796782 registros originales, objetivo 10662000
    Aumentando mediante duplicación a 10662000 registros
  - backdoor: 1389 registros originales, objetivo 30000
    Aumentando mediante duplicación a 30000 registros
  - ddos_http: 43389 registros originales, objetivo 240000
    Aumentando mediante duplicación a 24000

**Comprobar todas las columnas del dataset final**

In [27]:
print("\nListado completo de características en el dataset :")
for i, column in enumerate(ng_iiotset_df.columns):
    print(f"{i+1}. {column}")


Listado completo de características en el dataset :
1. uid
2. ts
3. id.orig_h
4. id.resp_h
5. id.resp_p
6. proto
7. service
8. conn_state
9. duration
10. orig_bytes
11. resp_bytes
12. orig_pkts
13. resp_pkts
14. ip_proto
15. isAttack
16. typeAttack
17. query
18. answers
19. qtype_name
20. rcode
21. rcode_name
22. connect_status
23. client_id
24. topic
25. payload
26. func
27. pdu_type
28. exception
29. method
30. uri
31. user_agent
32. host
33. request_body_len
34. response_body_len
35. status_code
36. fuid
37. source
38. mime_type
39. filename
40. seen_bytes
41. total_bytes
42. md5
43. sha1
44. sha256
45. name
